    221028_exnode13_artist_recomm
        1. package load
        2. MovieLens data load
            - movieLens(ratings), movie title(movies) load
        3. data 전처리 및 확인 
            - data merge, column 조정
        4. data 탐색 
            - user, movie, genre 탐색
        5. model setting 
            - 초기 정보 setting : indexing(user,movie,genre)
        6. model creation
            - 2개 model 생성(1:user,title , 2:user,genre)
        7. model testing : model1(user.title)
            - 비슷한 영화 찾기, user에게 영화 추천하기
        8 model testing  : model2(user.genre) ... 무의미 할지도 모르지만 .. 확인 차원에서 실시
            - 비슷한 영화 찾기, user에게 영화 추천하기
    -----------------------------------------------------------------------------------------------------------------            
        9. 회고  
           - model1(user,title), model2(user,genre) CSR matrix가 정상적으로 만들어졌으며 
             model1(6039,),model2() 생성됨
           - 생성한 user와 영화에 대한 벡터 내적은 0.79로 의미 있는 관계가 형성되었다고 판담됨 
             생성한 user와 쟝르에 대한 벡터 내적은 0.8로 의미 있는 관계가 형성되었다고 판담됨 
           - 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
             . 기존 영화에 대한 유저 선호도는 높게 출현되었으며, 영화를 기준으로 비슷한 영화가 표시되었슴
               또한 기존의 시청영화가 추천영화에 기여한 정도를 표현하였다 
        -------------------------------------------------------------------------------------------------------------       
           - 2개의 모델 생성시 1개의 instance model에 모델을 훈련시킬수 없어, 별도의 instance 모델을 생성하여 
             커널 down을 극복하였슴.
           - user의 선호도, 추천영화, 추천영화에 대한 기존영화의 추천도가 생각보다 쉽게 산출할 수 있어 
             딥러닝을 통해 상업적인 부분에 많은 활용을 생각할 수 있겠다고 판단되었습니다.

In [179]:
# 1. package load
import numpy as np
import scipy
import implicit
import os
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


     2. MovieLens data load
        - movieLens(ratings), movie title(movies) load

In [180]:
# 2. MovieLens  data load .. movie ratings data
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [181]:
# 2. movie data load ... ## movie title 
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
print(movies['movie_id'].nunique())
movies.head(20)    # 3883(movies) - 3706(ratings movie) =  177

3883


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


    3. data 전처리 및 확인 
        - data merge : ratings data + title data
        - data 조정 : timestamp 제외 
        - ratings 조정 : 3점이상, columns 변경(ratings > counts)

In [182]:
# 3. data 전처리 및 확인  .. movie ratings data + movie title data
ratings = pd.merge(ratings, movies[['title','genre','movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [183]:
#3. data 전처리 및 확인 ... data 조정 : timestamp 제외
using_cols = ['user_id', 'movie_id', 'ratings','title','genre']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,ratings,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [184]:
#3. data 전처리 및 확인  ... 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [185]:
#3. data 전처리 및 확인  ... ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True) 
ratings['counts']
ratings.head(10)

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [186]:
#3. data 전처리 및 확인 ..  # title, genre object형 type
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836478 non-null  int64 
 1   movie_id  836478 non-null  int64 
 2   counts    836478 non-null  int64 
 3   title     836478 non-null  object
 4   genre     836478 non-null  object
dtypes: int64(3), object(2)
memory usage: 38.3+ MB


In [187]:
#3. data 전처리 및 확인 ... 첫 번째 유저가 영화 시청 항목
condition = (ratings['user_id']== ratings.loc[0, 'user_id'])
ratings.loc[condition]

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


    4. data 탐색 
        - user, movie, genre
        - 인기 영화 
        - user별 영화 시청 갯수 

In [188]:
#4. data 탐색  , genre
print("user    : ", ratings['user_id'].nunique())     # unique user
print("movie no: ", ratings['movie_id'].nunique())    # uniq movie
print("genre   : ", movies['genre'].nunique())       # uniq genre

user    :  6039
movie no:  3628
genre   :  301


In [189]:
#4. data 탐색   ,, top movie
# movie_count 인기 많은 movie
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(10)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
Name: user_id, dtype: int64

In [190]:
#4. data 탐색    ,, user별 영화 시청 

user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

    5. model setting  
        - user 생성
        - 초기 정보 setting(user,movie,genre indexing)
        - csr matrix creation : 별도 생성(csr

In [191]:
#5. model setting    ,, user 생성  
my_favorite_title = ['Toy Story (1995)','Jumanji (1995)','Grumpier Old Men (1995)','Waiting to Exhale (1995)','Father of the Bride Part II (1995)']
my_favorite_id = [1,2,3,4,5]
my_favorite_genre = ["Animation|Children's|Comedy","Adventure|Children's|Fantasy","Comedy|Romance","Comedy|Drama","Comedy"]
# '6041'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'movie_id': my_favorite_id,'counts':4,'title': my_favorite_title, 'genre':my_favorite_genre})

if not ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,zimin,1,4,Toy Story (1995),Animation|Children's|Comedy
1,zimin,2,4,Jumanji (1995),Adventure|Children's|Fantasy
2,zimin,3,4,Grumpier Old Men (1995),Comedy|Romance
3,zimin,4,4,Waiting to Exhale (1995),Comedy|Drama
4,zimin,5,4,Father of the Bride Part II (1995),Comedy


In [192]:
#5. model setting ...  # user, movie,genre를 찾아내는 코드
user_unique = ratings['user_id'].unique()
#movie_unique = ratings['movie_id'].unique()
title_unique = ratings['title'].unique()
genre_unique = ratings['genre'].unique()
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
#movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}
genre_to_idx = {v:k for k,v in enumerate(genre_unique)}

In [193]:
#5. model setting   # 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6040이 나와야 합니다. 
#print(movie_to_idx[5])
print(title_to_idx['Waiting to Exhale (1995)'])
print(genre_to_idx['Comedy'])

6039
397
9


In [194]:
#5. model setting ... # indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
#temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
#if len(temp_movie_data) == len(ratings):
#    print('movie column indexing OK!!')
#    ratings['movie_id'] = temp_movie_data
#else:
#    print('movie column indexing Fail!!')
    
# title_to_idx 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다.     
temp_title_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')
    
# genre_to_idx 통해 genre 컬럼도 동일한 방식으로 인덱싱해 줍니다.       
temp_genre_data = ratings['genre'].map(genre_to_idx.get).dropna()
if len(temp_genre_data) == len(ratings):
    print('genre column indexing OK!!')
    ratings['genre'] = temp_genre_data
else:
    print('genre column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!
genre column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,0
1,0,661,3,1,1
2,0,914,3,2,2
3,0,3408,4,3,0
4,0,2355,5,4,3
...,...,...,...,...,...
0,6039,1,4,40,3
1,6039,2,4,513,134
2,6039,3,4,1862,45
3,6039,4,4,397,6


In [195]:
#5. model setting   # 2평점 항목 한 데이터의 비율을 보는 코드
only_one = ratings[ratings['counts']<3]
one, all_data = len(only_one), len(ratings)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


In [196]:
#5. model setting  ...  csr matrix 생성 (user, title, genre )
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_user1 = ratings['user_id'].nunique()
#num_movie = ratings['movie_id'].nunique()
num_title = ratings['title'].nunique()
num_genre = ratings['genre'].nunique()

#csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data1 = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_title))
csr_data2 = csr_matrix((ratings.counts, (ratings.user_id, ratings.genre)), shape= (num_user1, num_genre))
print(csr_data1)
print(csr_data2)

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 40)	4
  (6039, 397)	4
  (6039, 513)	4
  (6039, 1180)	4
  (6039, 1862)	4
  (0, 0)	53
  (0, 1)	26
  (0, 2)	3
  (0, 3)	14
  (0, 4)	3
  (0, 5)	5
  (0, 6)	5
  (0, 7)	4
  (0, 8)	9
  (0, 9)	11
  (0, 10)	15
  (0, 11)	4
  (0, 12)	3
  (0, 13)	5
  (0, 14)	5
  (0, 15)	3
  (0, 16)	5
  (0, 17)	4
  (0, 18)	4
  (0, 19)	3
  (0, 20)	4
  (0, 21)	4
  (0, 22)	4
  (0, 23)	4


    6. model creation  
        - parameter setting
        - 2개 model 생성(1:user,title , 2:user,genre)
          . model definition
          . model creation 
          . model trainning

In [197]:
#6. model creation  ... parameter setting
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [198]:
#6. model creation   # Implicit AlternatingLeastSquares 모델의 선언
als_model1 = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
als_model2 = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [199]:
#6. model creation  .... als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
#csr_data_transpose = csr_data.T
csr_data_transpose1 = csr_data1.T
csr_data_transpose2 = csr_data2.T

In [200]:
# 모델 훈련
#als_model.fit(csr_data_transpose)

In [201]:
#6. model creation ... 모델 훈련1(user.title)
als_model1.fit(csr_data_transpose1)

  0%|          | 0/15 [00:00<?, ?it/s]

In [202]:
#6. model creation  .. 모델 훈련2(user.genre)
als_model2.fit(csr_data_transpose2)

  0%|          | 0/15 [00:00<?, ?it/s]

    7 model testing
        - model1(user.title)
        - 비슷한 영화 찾기
        - user에게 영화 추천하기

In [203]:
#7 model testing ... model 확인 (user.title)
zimin, Balto_1995 = user_to_idx['zimin'], title_to_idx['Balto (1995)']
zimin_vector, Balto_1995_vector = als_model1.user_factors[zimin], als_model1.item_factors[Balto_1995]

print('슝=3')

슝=3


In [204]:
#7 model testing ... model 확인 (user)
zimin_vector

array([-0.17278603,  0.29009333,  0.19503559, -0.25600433,  0.10692843,
       -0.13423336,  0.72642046,  0.08072069, -0.20199327, -0.14781083,
       -0.2967389 , -0.3818326 ,  0.37072206,  0.71227425, -0.36262268,
        0.12938496,  0.4735556 ,  0.18620692, -0.04220046,  0.29550213,
        0.41728622, -0.17998694,  0.07472843,  0.47526583,  0.02366098,
       -0.62263507,  0.2301272 ,  0.14131773,  0.06324137, -0.40772364,
       -0.6939349 , -0.20588852, -0.48574844,  0.02594265,  0.29179186,
       -0.6240956 ,  0.15119627,  0.02980183,  0.13475937,  0.07321522,
        0.3174654 , -0.51702696, -0.22575617, -0.36781186, -0.5672796 ,
       -0.3213259 ,  0.4567986 ,  0.42504048, -0.04216471, -0.11563286,
        0.24654096, -0.2868112 , -0.10034736,  0.08811857, -0.3343124 ,
       -0.29202482, -0.41838932,  0.07035394, -0.18143435,  0.45450604,
       -0.2034456 , -0.43464187, -0.14117585,  0.83550423, -0.26114243,
       -0.8077773 , -0.34910196,  0.27327326, -0.15245333,  0.01

In [205]:
#7 model testing ... model 확인(title) 
Balto_1995_vector

array([-0.00352166, -0.0056825 , -0.00314997,  0.00375769,  0.00991932,
        0.00132418,  0.00697037,  0.01150056,  0.00546102, -0.00055937,
       -0.0132314 ,  0.00415193,  0.01195121,  0.01074868,  0.01068082,
        0.00274492,  0.00216104, -0.00093547,  0.00309598, -0.00293985,
        0.00409046,  0.00713702, -0.00720006, -0.00457626,  0.00386191,
       -0.00364039,  0.01427132,  0.0045719 , -0.0022361 ,  0.008285  ,
       -0.00010305, -0.01297785, -0.00429077,  0.00176257,  0.01118487,
        0.00273442,  0.0101581 ,  0.00174556, -0.00797248,  0.00225106,
        0.00648518,  0.00752893,  0.00695988,  0.0109168 ,  0.009109  ,
        0.01095635,  0.00083884,  0.00911211,  0.00398725,  0.00163845,
       -0.00433385,  0.01609593,  0.0031393 ,  0.01224483, -0.001204  ,
        0.0099758 ,  0.00712347,  0.00558198,  0.005238  ,  0.00807629,
        0.00030656,  0.00235517, -0.00910854,  0.01437072, -0.00060231,
        0.01020657,  0.01151207,  0.01368336,  0.01595167, -0.00

In [206]:
#7 model testing ... model 확인(user.title)  .. zimin과 Balto_1995를 내적하는 코드
np.dot(zimin_vector, Balto_1995_vector)

0.02769629

In [207]:
#7 model testing .. model1(user.title)
Toy = title_to_idx['Toy Story (1995)']
Toy_vector = als_model1.item_factors[Toy]
np.dot(zimin_vector, Toy_vector)

0.40598375

In [212]:
#7 model testing . model1(user.genre)  .. 비슷한 영화 찾기
favorite_title = 'Platoon (1986)'
title_id = title_to_idx[favorite_title]
similar_title = als_model1.similar_items(title_id, N=15)
similar_title

[(1030, 1.0000001),
 (1084, 0.6638825),
 (1370, 0.58912885),
 (968, 0.56238556),
 (783, 0.50591916),
 (1300, 0.48873636),
 (1365, 0.46594945),
 (988, 0.45869276),
 (3088, 0.45167485),
 (1811, 0.44935173),
 (728, 0.44612536),
 (1182, 0.4435527),
 (1723, 0.42163444),
 (1815, 0.41923493),
 (1412, 0.4079347)]

In [213]:
#7 model testing . model1(user.genre) ... 비슷한 영화 찾기 #title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Platoon (1986)',
 'Full Metal Jacket (1987)',
 'Killing Fields, The (1984)',
 'Last Emperor, The (1987)',
 'Deer Hunter, The (1978)',
 'Apocalypse Now (1979)',
 'Patton (1970)',
 'Glory (1989)',
 'Open Season (1996)',
 'Raging Bull (1980)',
 'Good Morning, Vietnam (1987)',
 'Red Dawn (1984)',
 'Birdy (1984)',
 'Pink Floyd - The Wall (1982)',
 'Right Stuff, The (1983)']

In [214]:
#7 model testing . model1(user.genre) ... 비슷한 영화 찾기 함수
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model1.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [215]:
#7 model testing . model1(user.genre) ... 비슷한 영화 찾기 실행
get_similar_title('Grumpier Old Men (1995)')

['Grumpier Old Men (1995)',
 'Grumpy Old Men (1993)',
 'Milk Money (1994)',
 "She's the One (1996)",
 'Nine Months (1995)',
 'Naked Gun 33 1/3: The Final Insult (1994)',
 'Forget Paris (1995)',
 'Santa Claus: The Movie (1985)',
 'Mirror Has Two Faces, The (1996)',
 'Vegas Vacation (1997)']

In [216]:
#7 model testing . model1(user.genre)   .. 유저에게 영화 추천하기
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model1.recommend(user, csr_data1, N=20, filter_already_liked_items=True)
title_recommended

[(50, 0.41596445),
 (545, 0.23441249),
 (4, 0.20977935),
 (33, 0.1980082),
 (458, 0.18735361),
 (322, 0.18567628),
 (812, 0.1796403),
 (596, 0.17942765),
 (507, 0.17702226),
 (1045, 0.17700413),
 (119, 0.15917294),
 (330, 0.15768564),
 (10, 0.15200827),
 (1982, 0.15188117),
 (1130, 0.14590147),
 (32, 0.14028956),
 (173, 0.13849834),
 (341, 0.13742445),
 (284, 0.13530654),
 (1514, 0.12794788)]

In [217]:
#7 model testing . model1(user.genre)   .. 유저에게 영화 추천하기
[idx_to_title[i[0]] for i in title_recommended]

['Toy Story 2 (1999)',
 'Santa Clause, The (1994)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Mask, The (1994)',
 'Babe (1995)',
 'Nutty Professor, The (1996)',
 'Hook (1991)',
 'Home Alone (1990)',
 'Grumpy Old Men (1993)',
 'Twister (1996)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Space Jam (1996)',
 'Indian in the Cupboard, The (1995)',
 'Hercules (1997)',
 'Dragonheart (1996)',
 'Mighty Ducks, The (1992)',
 'Nightmare Before Christmas, The (1993)',
 'Six Days Seven Nights (1998)']

In [220]:
#7 model testing . model1(user.genre)   .. 유저에게 영화 추천하기  ... 이 추천에 기여한 정도를 확인
Welcome = title_to_idx['Toy Story (1995)']
explain = als_model1.explain(user, csr_data1, itemid=Welcome)

In [221]:
#7 model testing . model1(user.genre)   .. 유저에게 영화 추천하기  ... 이 추천에 기여한 정도를 확인
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.3691033160289936),
 ('Jumanji (1995)', 0.02085410830239859),
 ('Waiting to Exhale (1995)', 0.017205642739955077),
 ('Father of the Bride Part II (1995)', -0.0020184128445471057),
 ('Grumpier Old Men (1995)', -0.0057042599144647346)]

    8 model testing  ... 무의미 할지도 모르지만 .. 확인 차원에서 실시
        - model2(user.title)
        - 비슷한 영화 찾기
        - user에게 영화 추천하기

In [163]:
#8 model testing... model 확인 (user, genre)
zimin, Comedy_genre = user_to_idx['zimin'], genre_to_idx['Comedy']
zimin_vector, Comedy_genre_vector = als_model2.user_factors[zimin], als_model2.item_factors[Comedy_genre]

print('슝=3')

슝=3


In [164]:
#8 model testing... model 확인 (user, genre) .. zimin과 Comedy 내적하는 코드
np.dot(zimin_vector, Comedy_genre_vector)

0.80083454

In [165]:
#8 model testing... model 확인 (user, genre) 
Comedy_drama_genre = genre_to_idx['Comedy|Drama']
Comedy_drama_genre_vector = als_model2.item_factors[Comedy_drama_genre]
np.dot(zimin_vector, Comedy_drama_genre_vector)

0.7749661

In [166]:
#8 model testing... model 확인 (user, genre)  .. 비슷한 genre 찾기
favorite_genre = 'Comedy'
genre_id = genre_to_idx[favorite_genre]
similar_genre = als_model2.similar_items(genre_id, N=15)
similar_genre

[(9, 1.0000001),
 (6, 0.9946834),
 (45, 0.97356653),
 (0, 0.96583104),
 (18, 0.8684048),
 (44, 0.77514315),
 (34, 0.37978756),
 (236, 0.1993333),
 (48, 0.18834433),
 (299, 0.17244896),
 (283, 0.16215324),
 (271, 0.16211519),
 (231, 0.15875664),
 (280, 0.15754488),
 (147, 0.15619726)]

In [167]:
#8 model testing... model 확인 (user, genre) ... 비슷한 영화 찾기 #title_to_idx를 뒤집어, index로부터 title 이름을 얻는 dict를 생성 
idx_to_genre= {v:k for k,v in genre_to_idx.items()}
[idx_to_genre[i[0]] for i in similar_genre]

['Comedy',
 'Comedy|Drama',
 'Comedy|Romance',
 'Drama',
 'Drama|Romance',
 'Drama|Thriller',
 'Action|Thriller',
 "Action|Adventure|Children's|Sci-Fi",
 'Action|Sci-Fi|Thriller',
 "Action|Adventure|Children's",
 "Adventure|Animation|Children's",
 "Animation|Children's|Fantasy|War",
 'Action|Adventure|Sci-Fi|Thriller|War',
 "Adventure|Children's|Comedy|Fantasy|Romance",
 "Children's|Sci-Fi"]

In [168]:
#8 model testing... model 확인 (user, genre) .. 비슷한 genre 찾기 함수
def get_similar_genre(genre_name: str):
    genre_id = genre_to_idx[genre_name]
    similar_genre = als_model2.similar_items(genre_id)
    similar_genre = [idx_to_genre[i[0]] for i in similar_genre]
    return similar_genre

print("슝=3")

슝=3


In [169]:
#8 model testing... model 확인 (user, genre)  ... 비슷한 genre 찾기 실행
get_similar_genre('Comedy')

['Comedy',
 'Comedy|Drama',
 'Comedy|Romance',
 'Drama',
 'Drama|Romance',
 'Drama|Thriller',
 'Action|Thriller',
 "Action|Adventure|Children's|Sci-Fi",
 'Action|Sci-Fi|Thriller',
 "Action|Adventure|Children's"]

In [170]:
#8 model testing... model 확인 (user, genre)  .. 유저에게 genre 추천하기
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model2.recommend(user, csr_data2, N=20, filter_already_liked_items=True)
title_recommended

[(0, 0.74694204),
 (18, 0.60617495),
 (44, 0.48530334),
 (144, 0.24036118),
 (73, 0.21184495),
 (141, 0.20160282),
 (181, 0.18965384),
 (34, 0.18157515),
 (24, 0.17757833),
 (163, 0.17623836),
 (53, 0.16218714),
 (10, 0.15898256),
 (61, 0.15631106),
 (140, 0.14377564),
 (13, 0.14335349),
 (127, 0.13419174),
 (241, 0.12404559),
 (30, 0.12258843),
 (72, 0.118115686),
 (152, 0.11073242)]

In [171]:
#8 model testing... model 확인 (user, genre)    .. 유저에게 영화 추천하기
[idx_to_genre[i[0]] for i in title_recommended]

['Drama',
 'Drama|Romance',
 'Drama|Thriller',
 "Children's|Comedy|Fantasy",
 'Action|Adventure|Fantasy',
 "Adventure|Children's",
 'Adventure|Fantasy|Romance',
 'Action|Thriller',
 'Action|Adventure|Fantasy|Sci-Fi',
 "Adventure|Children's|Comedy|Fantasy",
 'Drama|Romance|War',
 "Animation|Children's",
 'Comedy|Horror',
 "Children's|Fantasy|Sci-Fi",
 'Comedy|Sci-Fi',
 'Comedy|Crime|Fantasy',
 "Adventure|Children's|Drama",
 'Action|Adventure|Mystery',
 'Adventure|Thriller',
 'Adventure|Fantasy']

In [172]:
#8 model testing... model 확인 (user, genre)   .. 유저에게 영화 추천하기  ... 이 추천에 기여한 정도를 확인
Welcome = genre_to_idx['Drama']
explain = als_model2.explain(user, csr_data2, itemid=Welcome)

In [173]:
#8 model testing... model 확인 (user, genre)   .. 유저에게 영화 추천하기  ... 이 추천에 기여한 정도를 확인
[(idx_to_genre[i[0]], i[1]) for i in explain[1]]

[('Comedy', 0.2553854007374663),
 ('Comedy|Drama', 0.2501127532255761),
 ('Comedy|Romance', 0.2307703414704475),
 ("Adventure|Children's|Fantasy", 0.0034372634625129926),
 ("Animation|Children's|Comedy", -0.007465324580659947)]